In [146]:
import numpy as np
import pandas as pd
from __future__ import print_function

from sklearn.ensemble  import RandomForestClassifier as rfc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [147]:
# the directory that contains all the files
dataDirectory = 'data/'

In [148]:
file = 'llcp2017_formatted.csv'
#df = pd.read_csv(dataDirectory+file)
df = pd.read_csv(file)

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (3,170,174) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [149]:
df.head()

,STATE FIPS CODE,FILE MONTH,INTERVIEW DATE,INTERVIEW MONTH,INTERVIEW DAY,INTERVIEW YEAR,FINAL DISPOSITION,ANNUAL SEQUENCE NUMBER,PRIMARY SAMPLING UNIT,CORRECT TELEPHONE NUMBER?,...,_IMPCAGE,_IMPCRAC,_IMPCSEX,_IMPEDUC,_IMPHOME,_IMPMRTL,_IMPNPH,_IMPSEX,_M_RACE,_URBNRRL
0,Alabama,January,1302017,01,30,2017,1100,2017000001,2017000001,Yes,...,Not asked or Missing,Missing,Missing,Missing,Missing,Missing,NaN,Not asked or Missing,Not asked or Missing,Missing
1,Alabama,January,1122017,01,12,2017,1100,2017000002,2017000002,Yes,...,Not asked or Missing,Missing,Missing,Missing,Missing,Missing,NaN,Not asked or Missing,Not asked or Missing,Missing
2,Alabama,January,1102017,01,10,2017,1100,2017000003,2017000003,Yes,...,Not asked or Missing,Missing,Missing,Missing,Missing,Missing,NaN,Not asked or Missing,Not asked or Missing,Missing
3,Alabama,January,2082017,02,8,2017,1200,2017000004,2017000004,Yes,...,Not asked or Missing,Missing,Missing,Missing,Missing,Missing,NaN,Not asked or Missing,Not asked or Missing,Missing
4,Alabama,January,1302017,01,30,2017,1100,2017000005,2017000005,Yes,...,Not asked or Missing,Missing,Missing,Missing,Missing,Missing,NaN,Not asked or Missing,Not asked or Missing,Missing


In [150]:
print("Number of records: ",len(df))

Number of records:  450642


In [151]:
df['INTERVIEW YEAR'].value_counts()/np.sum(df['INTERVIEW YEAR'].value_counts())

2017    0.973964
2018    0.026036
Name: INTERVIEW YEAR, dtype: float64

#### Most of the data is from 2017

In [152]:
list(np.sort(df.columns))

['(EVER TOLD) YOU HAD ANY OTHER TYPES OF C',
 '(EVER TOLD) YOU HAD SKIN CANCER?',
 '(EVER TOLD) YOU HAVE (COPD) CHRONIC OBST',
 '(EVER TOLD) YOU HAVE DIABETES',
 '(EVER TOLD) YOU HAVE KIDNEY DISEASE?',
 '150 MINUTE PHYSICAL ACTIVITY CALCULATED',
 '300 MINUTE PHYSICAL ACTIVITY 2-LEVEL CAL',
 '300 MINUTE PHYSICAL ACTIVITY CALCULATED',
 'ACTIVITIES LIMITED DUE TO ASTHMA DURING',
 'ACTIVITY MET VALUE FOR FIRST ACTIVITY',
 'ACTIVITY MET VALUE FOR SECOND ACTIVITY',
 'ADLTCHLD',
 'ADULT FLU SHOT/SPRAY PAST 12 MOS',
 'ADULTS WITH GOOD OR BETTER HEALTH',
 'AEROBIC AND STRENGTHENING (2-LEVEL)',
 'AEROBIC AND STRENGTHENING GUIDELINE',
 'AGE',
 'AGE AT ASTHMA DIAGNOSIS',
 'AGE TOLD HAD CANCER',
 'AGE WHEN TOLD DIABETIC',
 'ALWAYS OR NEARLY ALWAYS WEAR SEAT BELTS',
 'ALWAYS WEAR SEAT BELTS',
 'ANNUAL SEQUENCE NUMBER',
 'ANY FIREARMS IN HOME',
 'ANY FIREARMS LOADED',
 'ANY LOADED FIREARMS ALSO UNLOCKED',
 'ARE YOU 18 YEARS OF AGE OR OLDER?',
 'ARE YOU 18 YEARS OF AGE OR OLDER?.1',
 'ARE YOU A VETERA

In [153]:
# Find the columns that contain smoking key word

filterList = ['smok','cig']
def containsKeyword(sent):
    sent = sent.lower()
    for f in filterList:
        if f in sent:
            return True
    return False
    
smokecols = [c for c in df.columns if containsKeyword(c)]
smokecols

['SMOKED AT LEAST 100 CIGARETTES',
 'FREQUENCY OF DAYS NOW SMOKING',
 'STOPPED SMOKING IN PAST 12 MONTHS',
 'INTERVAL SINCE LAST SMOKED',
 'USE OF SMOKELESS TOBACCO PRODUCTS',
 'HAVE YOU EVER USED AN E-CIGARETTE OR OTH',
 'DO YOU NOW USE E-CIGARETTES, EVERY DAY,',
 'HOW MANY YEARS HAVE YOU SMOKED TOBACCO P',
 'HOW OLD WHEN YOU FIRST STARTED SMOKING?',
 'HOW OLD WHEN YOU LAST SMOKED?',
 'ON AVERAGE, HOW MANY CIGARETTES DO YOU S',
 'COMPUTED SMOKING STATUS',
 'CURRENT SMOKING CALCULATED VARIABLE',
 'COMPUTED E-CIGARETTE USER STATUS',
 'CURRENT E-CIGARETTE USER CALCULATED VARI']

In [154]:
# See some values in smoking columns
df[smokecols][:5]

,SMOKED AT LEAST 100 CIGARETTES,FREQUENCY OF DAYS NOW SMOKING,STOPPED SMOKING IN PAST 12 MONTHS,INTERVAL SINCE LAST SMOKED,USE OF SMOKELESS TOBACCO PRODUCTS,HAVE YOU EVER USED AN E-CIGARETTE OR OTH,"DO YOU NOW USE E-CIGARETTES, EVERY DAY,",HOW MANY YEARS HAVE YOU SMOKED TOBACCO P,HOW OLD WHEN YOU FIRST STARTED SMOKING?,HOW OLD WHEN YOU LAST SMOKED?,"ON AVERAGE, HOW MANY CIGARETTES DO YOU S",COMPUTED SMOKING STATUS,CURRENT SMOKING CALCULATED VARIABLE,COMPUTED E-CIGARETTE USER STATUS,CURRENT E-CIGARETTE USER CALCULATED VARI
0,No,Not asked or Missing,Not asked or Missing,Not asked or Missing,Not at all,No,Not asked or Missing,Not asked or Missing,Not asked or Missing,Not asked or Missing,Not asked or Missing,Never smoked,No,Never used E-cigarettes,Not currently using E-cigarettes
1,No,Not asked or Missing,Not asked or Missing,Not asked or Missing,Not at all,No,Not asked or Missing,Not asked or Missing,Not asked or Missing,Not asked or Missing,Not asked or Missing,Never smoked,No,Never used E-cigarettes,Not currently using E-cigarettes
2,No,Not asked or Missing,Not asked or Missing,Not asked or Missing,Not at all,No,Not asked or Missing,Not asked or Missing,Not asked or Missing,Not asked or Missing,Not asked or Missing,Never smoked,No,Never used E-cigarettes,Not currently using E-cigarettes
3,No,Not asked or Missing,Not asked or Missing,Not asked or Missing,Not at all,No,Not asked or Missing,Not asked or Missing,Not asked or Missing,Not asked or Missing,Not asked or Missing,Never smoked,No,Never used E-cigarettes,Not currently using E-cigarettes
4,Yes,Not at all,Not asked or Missing,Within the past month (less than 1 month ago),Not at all,No,Not asked or Missing,Not asked or Missing,Not asked or Missing,Not asked or Missing,Not asked or Missing,Former smoker,No,Never used E-cigarettes,Not currently using E-cigarettes


In [155]:
# See the frequency distribution of smoking columns
for c in smokecols:
    print(c)
    print(df[c].value_counts())
    print()

SMOKED AT LEAST 100 CIGARETTES
No                      246459
Yes                     185837
Not asked or Missing     15962
Don�t know/Not Sure       1938
Refused                    446
Name: SMOKED AT LEAST 100 CIGARETTES, dtype: int64

FREQUENCY OF DAYS NOW SMOKING
Not asked or Missing    264878
Not at all              121867
Every day                44680
Some days                18882
Refused                    237
Don�t Know/Not Sure         98
Name: FREQUENCY OF DAYS NOW SMOKING, dtype: int64

STOPPED SMOKING IN PAST 12 MONTHS
Not asked or Missing    387131
Yes                      35498
No                       27796
Don�t know/Not Sure        178
Refused                     39
Name: STOPPED SMOKING IN PAST 12 MONTHS, dtype: int64

INTERVAL SINCE LAST SMOKED
Not asked or Missing                                              328840
10 years or more                                                   80302
Within the past 5 years (1 year but less than 5 years ago)         16240
Withi

### The computed and calculated columns are cleaned data and have fewer missing values

In [156]:
## Explore the calculated and computed columns
filterList = ['computed', 'calculated']
def containsKeyword(sent):
    sent = sent.lower()
    for f in filterList:
        if f in sent:
            return True
    return False
    
cols = [c for c in df.columns if containsKeyword(c)]
print("Number of calculated/computed columns: ",len(cols))
cols

Number of calculated/computed columns:  41


['COMPUTED PHYSICAL HEALTH STATUS',
 'COMPUTED MENTAL HEALTH STATUS',
 'HIGH BLOOD PRESSURE CALCULATED VARIABLE',
 'CHOLESTEROL CHECKED CALCULATED VARIABLE',
 'HIGH CHOLESTEROL CALCULATED VARIABLE',
 'LIFETIME ASTHMA CALCULATED VARIABLE',
 'CURRENT ASTHMA CALCULATED VARIABLE',
 'COMPUTED ASTHMA STATUS',
 'COMPUTED PREFERRED RACE',
 'CALCULATED NON-HISPANIC RACE INCLUDING M',
 'COMPUTED RACE-ETHNICITY GROUPING',
 'COMPUTED NON-HISPANIC WHITES/ALL OTHERS',
 'COMPUTED FIVE LEVEL RACE/ETHNICITY CATEG',
 'COMPUTED RACE GROUPS USED FOR INTERNET P',
 'COMPUTED HEIGHT IN INCHES',
 'COMPUTED HEIGHT IN METERS',
 'COMPUTED WEIGHT IN KILOGRAMS',
 'COMPUTED BODY MASS INDEX',
 'COMPUTED BODY MASS INDEX CATEGORIES',
 'OVERWEIGHT OR OBESE CALCULATED VARIABLE',
 'COMPUTED NUMBER OF CHILDREN IN HOUSEHOLD',
 'COMPUTED LEVEL OF EDUCATION COMPLETED CA',
 'COMPUTED INCOME CATEGORIES',
 'COMPUTED SMOKING STATUS',
 'CURRENT SMOKING CALCULATED VARIABLE',
 'COMPUTED E-CIGARETTE USER STATUS',
 'CURRENT E-CIGARET

### These 41 columns calculated columns cover most socieconomic features so they should be used for initial models

In [157]:
for c in cols:
    print(c)
    print(df[c].value_counts())
    print()

COMPUTED PHYSICAL HEALTH STATUS
Zero days when physical health not good    276509
1-13 days when physical health not good    103480
14+ days when physical health not good      61308
Don�t know/Refused/Missing                   9345
Name: COMPUTED PHYSICAL HEALTH STATUS, dtype: int64

COMPUTED MENTAL HEALTH STATUS
Zero days when mental health not good    300525
1-13 days when mental health not good     93057
14+ days when mental health not good      49846
Don�t know/Refused/Missing                 7214
Name: COMPUTED MENTAL HEALTH STATUS, dtype: int64

HIGH BLOOD PRESSURE CALCULATED VARIABLE
No                                     267905
Yes                                    181337
Don�t know/Not Sure/Refused/Missing      1400
Name: HIGH BLOOD PRESSURE CALCULATED VARIABLE, dtype: int64

CHOLESTEROL CHECKED CALCULATED VARIABLE
Had cholesterol checked in past 5 years             385747
Have never had cholesterol checked                   26532
Don�t know/Not Sure Or Refused/Missing       

No                            368326
Yes                            63562
Don�t know/Refused/Missing     18754
Name: CURRENT SMOKING CALCULATED VARIABLE, dtype: int64

COMPUTED E-CIGARETTE USER STATUS
Never used E-cigarettes                      363247
Former E-cigarette user                       53028
Don�t know/Refused/Missing                    20678
Current E-cigarette user - uses some days      8667
Current E-cigarette user - uses every day      5022
Name: COMPUTED E-CIGARETTE USER STATUS, dtype: int64

CURRENT E-CIGARETTE USER CALCULATED VARI
Not currently using E-cigarettes    416275
Don�t know/Refused/Missing           20678
Current E-cigarette user             13689
Name: CURRENT E-CIGARETTE USER CALCULATED VARI, dtype: int64

COMPUTED DRINK-OCCASIONS-PER-DAY
Drink-Occasions per day                   221510
No Drink-Occasions per day                203392
Don�t know/Not Sure Or Refused/Missing     25740
Name: COMPUTED DRINK-OCCASIONS-PER-DAY, dtype: int64

BINGE DRINKING CALC

Age Less Than 65                          287849
Yes                                       107390
No                                         33679
Don�t know/Not Sure Or Refused/Missing     21724
Name: PNEUMONIA VACCINATION CALCULATED VARIABL, dtype: int64

EVER BEEN TESTED FOR HIV CALCULATED VARI
No                             268409
Yes                            127137
Not asked or missing            41727
Don�t know/Not Sure/Refused     13369
Name: EVER BEEN TESTED FOR HIV CALCULATED VARI, dtype: int64



In [158]:
# create a datframe that only has calculated columns
comDF = df[cols]
comDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450642 entries, 0 to 450641
Data columns (total 41 columns):
COMPUTED PHYSICAL HEALTH STATUS             450642 non-null object
COMPUTED MENTAL HEALTH STATUS               450642 non-null object
HIGH BLOOD PRESSURE CALCULATED VARIABLE     450642 non-null object
CHOLESTEROL CHECKED CALCULATED VARIABLE     450642 non-null object
HIGH CHOLESTEROL CALCULATED VARIABLE        450642 non-null object
LIFETIME ASTHMA CALCULATED VARIABLE         450642 non-null object
CURRENT ASTHMA CALCULATED VARIABLE          450642 non-null object
COMPUTED ASTHMA STATUS                      450642 non-null object
COMPUTED PREFERRED RACE                     450642 non-null object
CALCULATED NON-HISPANIC RACE INCLUDING M    450642 non-null object
COMPUTED RACE-ETHNICITY GROUPING            450642 non-null object
COMPUTED NON-HISPANIC WHITES/ALL OTHERS     450642 non-null object
COMPUTED FIVE LEVEL RACE/ETHNICITY CATEG    450642 non-null object
COMPUTED RACE GROU

In [159]:
print(pd.Series.idxmin(comDF['COMPUTED PHYSICAL HEALTH STATUS'].value_counts()))
comDF['COMPUTED PHYSICAL HEALTH STATUS'].value_counts()

Don�t know/Refused/Missing


Zero days when physical health not good    276509
1-13 days when physical health not good    103480
14+ days when physical health not good      61308
Don�t know/Refused/Missing                   9345
Name: COMPUTED PHYSICAL HEALTH STATUS, dtype: int64

In [160]:
comDF.describe()

,COMPUTED WEIGHT IN KILOGRAMS,COMPUTED FRUIT JUICE INTAKE IN TIMES PER,COMPUTED FRUIT INTAKE IN TIMES PER DAY,COMPUTED DARK GREEN VEGETABLE INTAKE IN,COMPUTED POTATO SERVINGS PER DAY,COMPUTED OTHER VEGETABLE INTAKE IN TIMES
count,420723.000000,417359.000000,416996.000000,417323.000000,413900.000000,413348.000000
mean,8158.443192,33.795399,116.103440,61.060855,25.512600,100.917307
std,2069.914152,103.738742,170.791619,133.629245,64.430663,201.645847
min,2268.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6759.000000,0.000000,43.000000,14.000000,7.000000,43.000000
50%,7938.000000,7.000000,100.000000,43.000000,14.000000,100.000000
75%,9163.000000,43.000000,200.000000,100.000000,29.000000,100.000000
max,27500.000000,9900.000000,9900.000000,9900.000000,9900.000000,9900.000000


In [161]:
print("Data types and unique values")
[ (comDF[c].dtype.name,len(comDF[c].value_counts()))  for c in comDF.columns]

Data types and unique values


[('object', 4),
 ('object', 4),
 ('object', 3),
 ('object', 4),
 ('object', 3),
 ('object', 3),
 ('object', 3),
 ('object', 4),
 ('object', 10),
 ('object', 10),
 ('object', 9),
 ('object', 3),
 ('object', 6),
 ('object', 6),
 ('object', 2),
 ('object', 2),
 ('float64', 561),
 ('object', 2),
 ('object', 5),
 ('object', 3),
 ('object', 7),
 ('object', 5),
 ('object', 6),
 ('object', 5),
 ('object', 3),
 ('object', 5),
 ('object', 3),
 ('object', 3),
 ('object', 3),
 ('object', 3),
 ('object', 3),
 ('float64', 103),
 ('float64', 128),
 ('float64', 119),
 ('float64', 92),
 ('float64', 120),
 ('object', 4),
 ('object', 4),
 ('object', 4),
 ('object', 4),
 ('object', 4)]

In [162]:
print("NANs for numeric variables")
[ (c, np.sum(np.isnan(comDF[c])))  for c in comDF.columns if np.issubdtype(comDF[c].dtype, np.number) ]

NANs for numeric variables


[('COMPUTED WEIGHT IN KILOGRAMS', 29919),
 ('COMPUTED FRUIT JUICE INTAKE IN TIMES PER', 33283),
 ('COMPUTED FRUIT INTAKE IN TIMES PER DAY', 33646),
 ('COMPUTED DARK GREEN VEGETABLE INTAKE IN', 33319),
 ('COMPUTED POTATO SERVINGS PER DAY', 36742),
 ('COMPUTED OTHER VEGETABLE INTAKE IN TIMES', 37294)]

In [163]:
print("Records: ",len(comDF))
comDF = comDF.dropna()
print("Records after cleaning Nan: ",len(comDF))

Records:  450642
Records after cleaning Nan:  379592


In [164]:
print("Infintes for numeric variables")
[ (c, np.sum(np.isinf(comDF[c])))  for c in comDF.columns if np.issubdtype(comDF[c].dtype, np.number) ]

Infintes for numeric variables


[('COMPUTED WEIGHT IN KILOGRAMS', 0),
 ('COMPUTED FRUIT JUICE INTAKE IN TIMES PER', 0),
 ('COMPUTED FRUIT INTAKE IN TIMES PER DAY', 0),
 ('COMPUTED DARK GREEN VEGETABLE INTAKE IN', 0),
 ('COMPUTED POTATO SERVINGS PER DAY', 0),
 ('COMPUTED OTHER VEGETABLE INTAKE IN TIMES', 0)]

In [165]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
def labelEncodeCategoricalFeatures(DF):
    df = DF.copy()
    for c in df.columns:
        if df[c].dtype.name == 'object':
            df[c] = labelencoder.fit_transform(df[c])
    return df

In [21]:
# remove class imballance by sampling from majority class
def classImballanceDownSample(df,ycol):
    df = df.copy()
    valueCount = df[ycol].value_counts()
    print("Before Class Imballance Treatment: ")
    print(valueCount)
    classes = valueCount.index
    counts = valueCount.values
    minClassSize = np.min(counts)
    for clas in classes:
        df1 = df[df[ycol]==clas]
        df2 = df[df[ycol]!=clas]
        
        df1 = df1.sample(n=minClassSize, random_state=50)
        df = df1.append(df2)
    #shuffling the dataframe
    df = df.sample(frac=1).reset_index(drop=True)
    print("After Class Imballance Treatment: ")
    print(df[ycol].value_counts())
    return df  

In [22]:
# remove class imballance by up sampling minority classes
def classImballanceUpSample(df,ycol):
    df = df.copy()
    valueCount = df[ycol].value_counts()
    print("Before Class Imballance Treatment: ")
    print(valueCount)
    classes = valueCount.index
    counts = valueCount.values
    maxClassSize = np.max(counts).astype(float)
    fractions = maxClassSize / counts
    for clas,f in zip(classes,fractions):
        df1 = df[df[ycol]==clas]
        df2 = df[df[ycol]!=clas]
        
        df1 = df1.sample(frac=f, replace=True, random_state=50)
        df = df1.append(df2)
    #shuffling the dataframe
    df = df.sample(frac=1).reset_index(drop=True)
    print("After Class Imballance Treatment: ")
    print(df[ycol].value_counts())
    return df  

In [23]:
# model that trains a default random forest and prints the report
def getModel(df, ballanceClasses=None, rfcBalance = None):
    
    xcols = df.columns[:len(df.columns)-1]
    ycols = [df.columns[-1]]
    print("Predictors: ",len(xcols))
    print("label: ",ycols[0])
    
#     if ballanceClasses != None:
#         df = ballanceClasses(df, ycols[0])
        
    X = df[xcols]
    Y = df[ycols]
    
    X = labelEncodeCategoricalFeatures(X)
    print("Label Encoding.")
    
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42, shuffle=True)
    print("Train Test Split:")
    print(np.array(X_train).shape, np.array(X_test).shape, np.array(y_train).shape, np.array(y_test).shape)
    
    if ballanceClasses != None:
        dfTrain = X_train
        dfTrain['label'] = y_train
        dfTrain = ballanceClasses(dfTrain,'label')
        X_train = dfTrain[ [c for c in dfTrain.columns if c not in 'label'] ]
        y_train = dfTrain['label']
    
    #print("Checking if sampling worked:")
    #print(y_train.value_counts())
    #print()
    
    if rfcBalance == None:
        clf = rfc()
    else:
        clf = rfc(class_weight = rfcBalance)
        print(clf)
    
    #clf = rfc()
    clf.fit(X_train, y_train.values.ravel())
    y_pred = clf.predict(X_test)
    print("Accuracy : ", np.sum(y_test[ycols[0]].values==y_pred).astype(float)/len(y_test) )
    print(classification_report(y_test[ycols[0]].values, y_pred))

### Predicting current calculated variables

In [24]:
## Predicting current calculated variables
xcols = [c for c in cols if c not in smokecols] + ['CURRENT SMOKING CALCULATED VARIABLE']
doesSmokeDf = comDF[xcols]
print("Original: ",len(doesSmokeDf))
doesSmokeDf = doesSmokeDf[doesSmokeDf['CURRENT SMOKING CALCULATED VARIABLE'].apply(lambda x: x in ['Yes','No'])]
print("Cleaned Label: ",len(doesSmokeDf))

print("Columns :",len(doesSmokeDf.columns))
print()
print(doesSmokeDf['CURRENT SMOKING CALCULATED VARIABLE'].value_counts())

Original:  379592
Cleaned Label:  377922
Columns : 38

No     322389
Yes     55533
Name: CURRENT SMOKING CALCULATED VARIABLE, dtype: int64


In [25]:
## Without removing the class imballance
getModel(doesSmokeDf)

Predictors:  37
label:  CURRENT SMOKING CALCULATED VARIABLE
Label Encoding.
Train Test Split:
(253207, 37) (124715, 37) (253207, 1) (124715, 1)
Accuracy :  0.8521027943711662
             precision    recall  f1-score   support

         No       0.87      0.98      0.92    106363
        Yes       0.49      0.13      0.20     18352

avg / total       0.81      0.85      0.81    124715



### Class imballance is influencing the model to converge towards a biased model. We need to treat class imballance.

### Upsampling before test train split gives extremely high accuracy but it is not correct. Because a lot of training examples show up in test and thus we are not actually measuring out of sample performance. So I changed code to treat imballance only in the train data.

In [26]:
## With upsampling the minority class
getModel(doesSmokeDf, classImballanceUpSample)

Predictors:  37
label:  CURRENT SMOKING CALCULATED VARIABLE
Label Encoding.
Train Test Split:
(253207, 37) (124715, 37) (253207, 1) (124715, 1)


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Before Class Imballance Treatment: 
No     216026
Yes     37181
Name: label, dtype: int64
After Class Imballance Treatment: 
No     216026
Yes    216026
Name: label, dtype: int64
Accuracy :  0.8330834302209037
             precision    recall  f1-score   support

         No       0.88      0.93      0.90    106363
        Yes       0.40      0.27      0.32     18352

avg / total       0.81      0.83      0.82    124715



In [27]:
## With downsampling the majority class
getModel(doesSmokeDf, classImballanceDownSample)

Predictors:  37
label:  CURRENT SMOKING CALCULATED VARIABLE
Label Encoding.
Train Test Split:
(253207, 37) (124715, 37) (253207, 1) (124715, 1)
Before Class Imballance Treatment: 
No     216026
Yes     37181
Name: label, dtype: int64


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


After Class Imballance Treatment: 
Yes    37181
No     37181
Name: label, dtype: int64
Accuracy :  0.71474161087279
             precision    recall  f1-score   support

         No       0.92      0.73      0.81    106363
        Yes       0.28      0.61      0.39     18352

avg / total       0.82      0.71      0.75    124715



### COMPUTED SMOKING STATUS

In [28]:
## Predicting current calculated variables
xcols = [c for c in cols if c not in smokecols] + ['COMPUTED SMOKING STATUS']
quitSmokeDf = comDF[xcols]
print("Original: ",len(quitSmokeDf))
quitSmokeDf = quitSmokeDf[quitSmokeDf['COMPUTED SMOKING STATUS'].apply(lambda x: x in  \
                                                                       ['Former smoker', \
                                                                        'Current smoker - now smokes every day',\
                                                                      'Current smoker - now smokes some days'])]
quitSmokeDf['COMPUTED SMOKING STATUS'] = quitSmokeDf['COMPUTED SMOKING STATUS'].apply(lambda x: x if 'Current smoker' not in x \
                                                                                     else 'Current smoker')

print("Cleaned Label: ",len(quitSmokeDf))

print("Columns :",len(quitSmokeDf.columns))
print()
print(quitSmokeDf['COMPUTED SMOKING STATUS'].value_counts())

Original:  379592
Cleaned Label:  162967
Columns : 38

Former smoker     107434
Current smoker     55533
Name: COMPUTED SMOKING STATUS, dtype: int64


In [29]:
## Without removing the class imballance
getModel(quitSmokeDf)

Predictors:  37
label:  COMPUTED SMOKING STATUS
Label Encoding.
Train Test Split:
(109187, 37) (53780, 37) (109187, 1) (53780, 1)
Accuracy :  0.6981405727036073
                precision    recall  f1-score   support

Current smoker       0.55      0.55      0.55     18151
 Former smoker       0.77      0.78      0.77     35629

   avg / total       0.70      0.70      0.70     53780



In [30]:
## With upsampling the minority class
getModel(quitSmokeDf, classImballanceUpSample)

Predictors:  37
label:  COMPUTED SMOKING STATUS
Label Encoding.
Train Test Split:
(109187, 37) (53780, 37) (109187, 1) (53780, 1)
Before Class Imballance Treatment: 
Former smoker     71805
Current smoker    37382
Name: label, dtype: int64


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


After Class Imballance Treatment: 
Former smoker     71805
Current smoker    71805
Name: label, dtype: int64
Accuracy :  0.672201561918929
                precision    recall  f1-score   support

Current smoker       0.51      0.67      0.58     18151
 Former smoker       0.80      0.68      0.73     35629

   avg / total       0.70      0.67      0.68     53780



In [31]:
## With downsampling the majority class
getModel(quitSmokeDf, classImballanceDownSample)

Predictors:  37
label:  COMPUTED SMOKING STATUS
Label Encoding.
Train Test Split:
(109187, 37) (53780, 37) (109187, 1) (53780, 1)


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Before Class Imballance Treatment: 
Former smoker     71805
Current smoker    37382
Name: label, dtype: int64
After Class Imballance Treatment: 
Former smoker     37382
Current smoker    37382
Name: label, dtype: int64
Accuracy :  0.648214949795463
                precision    recall  f1-score   support

Current smoker       0.49      0.74      0.59     18151
 Former smoker       0.82      0.60      0.69     35629

   avg / total       0.71      0.65      0.66     53780



### Explore E ciggerete smoking

In [32]:
ecigdf = comDF.copy()
ecigdf.rename(columns={'CURRENT E-CIGARETTE USER CALCULATED VARI': 'esmoke', \
                     'CURRENT SMOKING CALCULATED VARIABLE':'smoke'},inplace = True)


mapper = {'Current E-cigarette user': 'Yes', 'Not currently using E-cigarettes': 'No' }
ecigdf['esmoke'] = ecigdf['esmoke'].map(mapper)

print("data size: ",len(ecigdf))
ecigdf = ecigdf[ecigdf.smoke.apply(lambda x: x in ['Yes','No']) ]
ecigdf = ecigdf[ecigdf.esmoke.apply(lambda x: x in ['Yes','No']) ]
print("clean data size: ",len(ecigdf))

print("Frequency distributions:")
print(ecigdf.esmoke.value_counts())
print(ecigdf.smoke.value_counts())

data size:  379592
clean data size:  377435
Frequency distributions:
No     365208
Yes     12227
Name: esmoke, dtype: int64
No     321970
Yes     55465
Name: smoke, dtype: int64


In [33]:
print("Just smoke :", np.sum((ecigdf.smoke == 'Yes') & (ecigdf.esmoke == 'No')))
print("Just esmoke:", np.sum((ecigdf.smoke == 'No') & (ecigdf.esmoke == 'Yes')))
print("SMoke Both :", np.sum((ecigdf.smoke == 'Yes') & (ecigdf.esmoke == 'Yes')))
print("Smoke none :", np.sum((ecigdf.smoke == 'No') & (ecigdf.esmoke == 'No')))

Just smoke : 48998
Just esmoke: 5760
SMoke Both : 6467
Smoke none : 316210


### Assumption: Considering esmokers and those who smoke both as esmokers.

In [145]:
# remove non smokers
ecigdf = ecigdf[ (ecigdf.smoke == 'Yes') | (ecigdf.esmoke == 'Yes') ]
print("clean data size: ",len(ecigdf))

AttributeError: 'DataFrame' object has no attribute 'smoke'

In [35]:
# label is 1 for esmoker and zero for non smoker
ecigdf['label'] = 0
ecigdf['label'] = 
int(ecigdf.esmoke == 'Yes') + int(ecigdf.smoke == 'Yes') 

In [36]:
excludecolumns = smokecols + ['esmoke', 'smoke']
ecigdf = ecigdf[[c for c in ecigdf.columns if c not in excludecolumns ]]
print('Columns : ',len(ecigdf.columns))
print("Frequency distributions:")
print(ecigdf.label.value_counts())

Columns :  38
Frequency distributions:
False    48998
True     12227
Name: label, dtype: int64


In [37]:
## Without removing the class imballance
getModel(ecigdf)

Predictors:  37
label:  label
Label Encoding.
Train Test Split:
(41020, 37) (20205, 37) (41020, 1) (20205, 1)
Accuracy :  0.7888146498391487
             precision    recall  f1-score   support

      False       0.80      0.98      0.88     16115
       True       0.33      0.04      0.07      4090

avg / total       0.70      0.79      0.72     20205



In [38]:
## With upsampling the minority class
getModel(ecigdf, classImballanceUpSample)

Predictors:  37
label:  label
Label Encoding.
Train Test Split:
(41020, 37) (20205, 37) (41020, 1) (20205, 1)
Before Class Imballance Treatment: 
False    32883
True      8137
Name: label, dtype: int64
After Class Imballance Treatment: 
True     32883
False    32883
Name: label, dtype: int64


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Accuracy :  0.7492699826775551
             precision    recall  f1-score   support

      False       0.81      0.90      0.85     16115
       True       0.29      0.17      0.22      4090

avg / total       0.71      0.75      0.72     20205



In [39]:
## With downsampling the majority class
getModel(ecigdf, classImballanceDownSample)

Predictors:  37
label:  label
Label Encoding.
Train Test Split:
(41020, 37) (20205, 37) (41020, 1) (20205, 1)
Before Class Imballance Treatment: 
False    32883
True      8137
Name: label, dtype: int64
After Class Imballance Treatment: 
True     8137
False    8137
Name: label, dtype: int64


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Accuracy :  0.6160356347438752
             precision    recall  f1-score   support

      False       0.83      0.65      0.73     16115
       True       0.25      0.46      0.33      4090

avg / total       0.71      0.62      0.65     20205



In [40]:
## Use rfc weight paramter to ballance “balanced”, “balanced_subsample”
weights = {True:4,False:1}
getModel(ecigdf, ballanceClasses = None, rfcBalance = weights)

Predictors:  37
label:  label
Label Encoding.
Train Test Split:
(41020, 37) (20205, 37) (41020, 1) (20205, 1)
RandomForestClassifier(bootstrap=True, class_weight={True: 4, False: 1},
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)
Accuracy :  0.7915367483296214
             precision    recall  f1-score   support

      False       0.80      0.98      0.88     16115
       True       0.35      0.03      0.06      4090

avg / total       0.71      0.79      0.72     20205



### Weights argument is not solving the imballance problem.

### Some Rules of Thumb for Imballance

    - Consider testing under-sampling when you have an a lot data (tens- or hundreds of thousands of instances or more)
    - Consider testing over-sampling when you don’t have a lot of data (tens of thousands of records or less)
    - Consider testing random and non-random (e.g. stratified) sampling schemes.
    - Consider testing different resampled ratios (e.g. you don’t have to target a 1:1 ratio in a binary classification problem, try other ratios)


### ALso try SMOTE

## Chi-Square

In [49]:
for i in ecigdf.columns:
    if i == 'smoke':
        print('yes')

In [43]:
ecigdf.head()

,COMPUTED PHYSICAL HEALTH STATUS,COMPUTED MENTAL HEALTH STATUS,HIGH BLOOD PRESSURE CALCULATED VARIABLE,CHOLESTEROL CHECKED CALCULATED VARIABLE,HIGH CHOLESTEROL CALCULATED VARIABLE,LIFETIME ASTHMA CALCULATED VARIABLE,CURRENT ASTHMA CALCULATED VARIABLE,COMPUTED ASTHMA STATUS,COMPUTED PREFERRED RACE,CALCULATED NON-HISPANIC RACE INCLUDING M,...,COMPUTED FRUIT INTAKE IN TIMES PER DAY,COMPUTED DARK GREEN VEGETABLE INTAKE IN,COMPUTED POTATO SERVINGS PER DAY,COMPUTED OTHER VEGETABLE INTAKE IN TIMES,150 MINUTE PHYSICAL ACTIVITY CALCULATED,300 MINUTE PHYSICAL ACTIVITY CALCULATED,FLU SHOT CALCULATED VARIABLE,PNEUMONIA VACCINATION CALCULATED VARIABL,EVER BEEN TESTED FOR HIV CALCULATED VARI,label
5,1-13 days when physical health not good,Zero days when mental health not good,Yes,Had cholesterol checked in past 5 years,No,No,No,Never,White,White only,...,2.0,29.0,43.0,100.0,1-149 minutes (or vigorous equivalent minutes...,1-300 minutes (or vigorous equivalent minutes...,Yes,No,No,False
20,14+ days when physical health not good,Zero days when mental health not good,Yes,Had cholesterol checked in past 5 years,Yes,No,No,Never,White,White only,...,300.0,7.0,7.0,10.0,0 minutes (or vigorous equivalent minutes) of ...,0 minutes (or vigorous equivalent minutes) of...,Yes,No,No,True
26,Zero days when physical health not good,Zero days when mental health not good,No,Have never had cholesterol checked,Missing,No,No,Never,White,White only,...,100.0,100.0,43.0,100.0,1-149 minutes (or vigorous equivalent minutes...,1-300 minutes (or vigorous equivalent minutes...,Age Less Than 65,Age Less Than 65,Yes,False
27,1-13 days when physical health not good,1-13 days when mental health not good,No,Had cholesterol checked in past 5 years,Yes,No,No,Never,White,White only,...,29.0,33.0,7.0,23.0,150+ minutes (or vigorous equivalent minutes) ...,301+ minutes (or vigorous equivalent minutes)...,No,Yes,No,False
29,1-13 days when physical health not good,Zero days when mental health not good,No,Did not have cholesterol checked in past 5 years,No,No,No,Never,White,White only,...,57.0,43.0,43.0,100.0,150+ minutes (or vigorous equivalent minutes) ...,1-300 minutes (or vigorous equivalent minutes...,Age Less Than 65,Age Less Than 65,No,False
34,Zero days when physical health not good,14+ days when mental health not good,No,Don�t know/Not Sure Or Refused/Missing,Missing,No,No,Never,White,White only,...,67.0,17.0,67.0,50.0,0 minutes (or vigorous equivalent minutes) of ...,0 minutes (or vigorous equivalent minutes) of...,Age Less Than 65,Age Less Than 65,No,False
48,1-13 days when physical health not good,Don�t know/Refused/Missing,Yes,Don�t know/Not Sure Or Refused/Missing,Missing,Don�t know/Not Sure Or Refused/Missing,Don�t know/Not Sure Or Refused/Missing,Don�t know/Not Sure Or Refused/Missing,White,White only,...,29.0,100.0,43.0,100.0,1-149 minutes (or vigorous equivalent minutes...,1-300 minutes (or vigorous equivalent minutes...,Yes,Yes,No,False
56,1-13 days when physical health not good,Zero days when mental health not good,No,Had cholesterol checked in past 5 years,No,No,No,Never,White,White only,...,100.0,71.0,14.0,214.0,Don�t know/Not Sure/Refused/Missing,Don�t know/Not Sure/Refused/Missing,Don�t know/Not Sure Or Refused/Missing,Don�t know/Not Sure Or Refused/Missing,Not asked or missing,False
59,Zero days when physical health not good,Zero days when mental health not good,Yes,Had cholesterol checked in past 5 years,No,No,No,Never,White,White only,...,100.0,29.0,14.0,100.0,0 minutes (or vigorous equivalent minutes) of ...,0 minutes (or vigorous equivalent minutes) of...,Age Less Than 65,Age Less Than 65,No,True
62,1-13 days when physical health not good,Zero days when mental health not good,Yes,Had cholesterol checked in past 5 years,No,No,No,Never,White,White only,...,0.0,14.0,13.0,13.0,Don�t know/Not Sure/Refused/Missing,Don�t know/Not Sure/Refused/Missing,Age Less Than 65,Age Less Than 65,No,False


In [136]:
ecigdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61225 entries, 5 to 450636
Data columns (total 39 columns):
COMPUTED PHYSICAL HEALTH STATUS             61225 non-null object
COMPUTED MENTAL HEALTH STATUS               61225 non-null object
HIGH BLOOD PRESSURE CALCULATED VARIABLE     61225 non-null object
CHOLESTEROL CHECKED CALCULATED VARIABLE     61225 non-null object
HIGH CHOLESTEROL CALCULATED VARIABLE        61225 non-null object
LIFETIME ASTHMA CALCULATED VARIABLE         61225 non-null object
CURRENT ASTHMA CALCULATED VARIABLE          61225 non-null object
COMPUTED ASTHMA STATUS                      61225 non-null object
COMPUTED PREFERRED RACE                     61225 non-null object
CALCULATED NON-HISPANIC RACE INCLUDING M    61225 non-null object
COMPUTED RACE-ETHNICITY GROUPING            61225 non-null object
COMPUTED NON-HISPANIC WHITES/ALL OTHERS     61225 non-null object
COMPUTED FIVE LEVEL RACE/ETHNICITY CATEG    61225 non-null object
COMPUTED RACE GROUPS USED FOR IN

In [143]:
from scipy.stats import chisquare



col_error= []
for i in [c for c in ecigdf.columns if c not in ['label']]:
    df = ecigdf
    column1 = 'label'
    column2 = i
    

    x_frac = df[df[column1] == True][[i,column1]].groupby(column2).count()[column1]
    y_frac = df[df[column1] == False][[i,column1]].groupby(column2).count()[column1]
    dof = 0
    try:
        print(chisquare(x_frac, f_exp=y_frac, ddof=dof, axis=0))
    except ValueError:
        print("Column %s is not available"%column2)
        col_error.append(column2)
print(col_error)

Power_divergenceResult(statistic=27622.659410497137, pvalue=0.0)
Power_divergenceResult(statistic=27694.626867832845, pvalue=0.0)
Power_divergenceResult(statistic=27684.10007522228, pvalue=0.0)
Power_divergenceResult(statistic=27692.41328702791, pvalue=0.0)
Power_divergenceResult(statistic=27705.310389108934, pvalue=0.0)
Power_divergenceResult(statistic=27624.188746688655, pvalue=0.0)
Power_divergenceResult(statistic=27603.847176388528, pvalue=0.0)
Power_divergenceResult(statistic=27650.395615394435, pvalue=0.0)
Power_divergenceResult(statistic=27687.15277140141, pvalue=0.0)
Power_divergenceResult(statistic=27702.150452436752, pvalue=0.0)
Power_divergenceResult(statistic=27698.23869679933, pvalue=0.0)
Power_divergenceResult(statistic=27611.540148286695, pvalue=0.0)
Power_divergenceResult(statistic=27667.129169192474, pvalue=0.0)
Power_divergenceResult(statistic=27667.129169192474, pvalue=0.0)
Power_divergenceResult(statistic=27597.371508038323, pvalue=0.0)
Power_divergenceResult(statis

In [144]:
df[df[column1] == False][[i,column1]].groupby(column2).count()

,label
test,
no,48998


In [140]:
col_error= []
for i in [c for c in ecigdf.columns if c not in ['label']]:
    df = ecigdf
    column1 = 'label'
    column2 = i
    

    x_frac = df[[column2,column1]].groupby(column1).count()[column2]
    y_frac = df[[column2,column1]].groupby(column2).count()[column1]
    dof = 0
    try:
        print(chisquare(x_frac, f_exp=y_frac, ddof=dof, axis=0))
    except ValueError:
        print("Column %s is not available"%column2)
        col_error.append(column2)

Column COMPUTED PHYSICAL HEALTH STATUS is not available
Column COMPUTED MENTAL HEALTH STATUS is not available
Column HIGH BLOOD PRESSURE CALCULATED VARIABLE is not available
Column CHOLESTEROL CHECKED CALCULATED VARIABLE is not available
Column HIGH CHOLESTEROL CALCULATED VARIABLE is not available
Column LIFETIME ASTHMA CALCULATED VARIABLE is not available
Column CURRENT ASTHMA CALCULATED VARIABLE is not available
Column COMPUTED ASTHMA STATUS is not available
Column COMPUTED PREFERRED RACE is not available
Column CALCULATED NON-HISPANIC RACE INCLUDING M is not available
Column COMPUTED RACE-ETHNICITY GROUPING is not available
Column COMPUTED NON-HISPANIC WHITES/ALL OTHERS is not available
Column COMPUTED FIVE LEVEL RACE/ETHNICITY CATEG is not available
Column COMPUTED RACE GROUPS USED FOR INTERNET P is not available
Power_divergenceResult(statistic=5312160.947323567, pvalue=0.0)
Power_divergenceResult(statistic=7997618.023571613, pvalue=0.0)
Column COMPUTED WEIGHT IN KILOGRAMS is not 

In [125]:
chisquare?

In [119]:
ecigdf.label.value_counts()

False    48998
True     12227
Name: label, dtype: int64

In [120]:
x_frac

array([8050, 4177])

In [122]:
df[df[column1] ==False][[i,column1]].groupby(column2).count()[column1]

test
no     33175
yes    15823
Name: label, dtype: int64

In [96]:
ecigdf[['COMPUTED WEIGHT IN KILOGRAMS', 
        'COMPUTED FRUIT JUICE INTAKE IN TIMES PER', 'COMPUTED FRUIT INTAKE IN TIMES PER DAY', 
        'COMPUTED DARK GREEN VEGETABLE INTAKE IN', 
        'COMPUTED POTATO SERVINGS PER DAY', 'COMPUTED OTHER VEGETABLE INTAKE IN TIMES']]

,COMPUTED WEIGHT IN KILOGRAMS,COMPUTED FRUIT JUICE INTAKE IN TIMES PER,COMPUTED FRUIT INTAKE IN TIMES PER DAY,COMPUTED DARK GREEN VEGETABLE INTAKE IN,COMPUTED POTATO SERVINGS PER DAY,COMPUTED OTHER VEGETABLE INTAKE IN TIMES
5,8890.0,7.0,2.0,29.0,43.0,100.0
20,10206.0,3.0,300.0,7.0,7.0,10.0
26,7484.0,100.0,100.0,100.0,43.0,100.0
27,5897.0,10.0,29.0,33.0,7.0,23.0
29,11340.0,43.0,57.0,43.0,43.0,100.0
34,4536.0,0.0,67.0,17.0,67.0,50.0
48,7257.0,29.0,29.0,100.0,43.0,100.0
56,4763.0,0.0,100.0,71.0,14.0,214.0
59,9979.0,100.0,100.0,29.0,14.0,100.0
62,10433.0,0.0,0.0,14.0,13.0,13.0


In [69]:
df = ecigdf
column1 = 'label'
df.groupby(column1).count()

,COMPUTED PHYSICAL HEALTH STATUS,COMPUTED MENTAL HEALTH STATUS,HIGH BLOOD PRESSURE CALCULATED VARIABLE,CHOLESTEROL CHECKED CALCULATED VARIABLE,HIGH CHOLESTEROL CALCULATED VARIABLE,LIFETIME ASTHMA CALCULATED VARIABLE,CURRENT ASTHMA CALCULATED VARIABLE,COMPUTED ASTHMA STATUS,COMPUTED PREFERRED RACE,CALCULATED NON-HISPANIC RACE INCLUDING M,...,COMPUTED FRUIT JUICE INTAKE IN TIMES PER,COMPUTED FRUIT INTAKE IN TIMES PER DAY,COMPUTED DARK GREEN VEGETABLE INTAKE IN,COMPUTED POTATO SERVINGS PER DAY,COMPUTED OTHER VEGETABLE INTAKE IN TIMES,150 MINUTE PHYSICAL ACTIVITY CALCULATED,300 MINUTE PHYSICAL ACTIVITY CALCULATED,FLU SHOT CALCULATED VARIABLE,PNEUMONIA VACCINATION CALCULATED VARIABL,EVER BEEN TESTED FOR HIV CALCULATED VARI
label,,,,,,,,,,,,,,,,,,,,,
False,48998,48998,48998,48998,48998,48998,48998,48998,48998,48998,...,48998,48998,48998,48998,48998,48998,48998,48998,48998,48998
True,12227,12227,12227,12227,12227,12227,12227,12227,12227,12227,...,12227,12227,12227,12227,12227,12227,12227,12227,12227,12227


In [ ]:
from scipy.stats import chisquare

def findchisqare

    chisquare(f_obs, f_exp=None, ddof=0, axis=0)[source]

In [ ]:
scipy.stats.chisquare?

** The chisq statistics tests the statistics calculated as : **

$\chi^2 = \sum_{i} \frac{(observation_i - expectation_i)^2}{expectation_i}$

 against a chi sq distribution.
 If we talk about sample fractions  that is 

$\chi^2 = \sum_i \frac{(f_{i,observed} - f_{i,expectated})^2}{f_{i,expected}}$

 Where _i_ indicates the sum over _each cell_.
 turns out this quantity is distributed according to a chi square distribution, so if i get the $\chi^2$ statistics i can compare it to the full chisq distribution and see how far in the tail it is

 The trickiest part (but not that tricky) is to figure out how to construct the table of values. please see Statistics In a Nutshell Chapter 4, for our data for example: Thisis called a CONTINGENCY TABLE